In [1]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

log_pr = pd.read_pickle("../data/log_price.df")
volu = pd.read_pickle("../data/volume_usd.df")

## Feature Engineering

In [40]:
features = log_pr.reset_index().melt(id_vars=['timestamp'])
features.columns = ['timestamp', 'stock', 'log_pr']
features = features.set_index(['timestamp', 'stock']).sort_index()

### backward returns

In [52]:
for i in [1, 3, 5, 10]:
    features['log_pr_{}'.format(i)] = features.groupby(level='stock').log_pr.diff(i)

### log volume and diffrence

In [41]:
log_volu = volu.apply(np.log).reset_index().melt(id_vars=['timestamp'])
log_volu.columns = ['timestamp', 'stock', 'log_volu']
log_volu = log_volu.set_index(['timestamp', 'stock'])

In [42]:
features['log_volu'] = log_volu['log_volu']

In [43]:
features.head()

log_pr   log_volu
timestamp  stock                     
2021-07-01 0      0.000051  11.901630
           1     -0.002754  13.272182
           2     -0.001690   9.830948
           3     -0.000106  11.983774
           4      0.002195  12.530313

In [48]:
for i in [1, 3, 5, 10]:
    features['log_volu_{}'.format(i)] = features.groupby(level='stock').log_volu.diff(i)